In [56]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
import xgboost as xgb

import numpy as np
import pandas as pd
import functools
from tqdm import tqdm

import stats_utils
import feature_utils
import processing

Чтение файлов

In [123]:
# Путь к папке с тренировочными данными
path_train = 'D:/Study/DS/Hacatons/AlfaHack/source/train'
# Путь к папке с тестовыми данными
path_test = 'D:/Study/DS/Hacatons/AlfaHack/source/test'

In [124]:
data_train = processing.read_files(path_train)
data_test = processing.read_files(path_test)

Балансировка классов

In [125]:
data_for_balanacing_X = data_train.copy().drop('target', axis=1)
data_for_balanacing_y = data_train['target']

In [126]:
df_balanced = processing.undersamle_data(data_for_balanacing_X, data_for_balanacing_y)

d:\Study\DS\Hacatons\AlfaHack\processing.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_resampled['target'] = y_resampled


Выделение похожих данных

In [127]:
condition = lambda df: (df % 1 != 0).any()
df_timeseries_train, remaining_data_train = processing.split_dataframe_by_condition(df_balanced, condition)
df_timeseries_test, remaining_data_test = processing.split_dataframe_by_condition(data_test, condition)

In [128]:
processing.df_processing(remaining_data_train, True)
processing.df_processing(remaining_data_test, True)

d:\Study\DS\Hacatons\AlfaHack\processing.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('int')
d:\Study\DS\Hacatons\AlfaHack\processing.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('int')
d:\Study\DS\Hacatons\AlfaHack\processing.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

Формирование необходимого для обработки датафрейма

In [129]:
data_train_transform = processing.df_transforming(df_timeseries_train)
data_test_transform = processing.df_transforming(df_timeseries_test)

  0%|          | 0/1686 [00:00<?, ?it/s]

100%|██████████| 19826/19826 [00:00<00:00, 6610196.43it/s]


In [130]:
time_series_data = processing.create_time_series_dict(data_train_transform)
time_series_data_test = processing.create_time_series_dict(data_test_transform)

100%|██████████| 19826/19826 [00:00<00:00, 38274.53it/s]


In [131]:
stats_features = {
    'mean': np.mean,
    'std': np.std,
    "min": np.min,
    "max": np.max,
    'max_diff': stats_utils.max_diff,
    '50%': np.median,
}

In [132]:
business_features = {
    "absolute_gain_basis": stats_utils.absolute_gain_basis,
    "growth_coef_basis": stats_utils.growth_coef_basis,
    "average_gain_rate": stats_utils.average_gain_rate,
    "average_growth_rate": stats_utils.average_growth_rate,
    "average_absolute_gain": stats_utils.average_absolute_gain,
}

In [157]:
X, y = feature_utils.construct_features_df_parallel(time_series_data, [
    functools.partial(feature_utils.get_fn_features, stats_features),
    functools.partial(feature_utils.get_fn_features, business_features),
    functools.partial(feature_utils.get_fourier_features, 15), 
    functools.partial(feature_utils.get_ma_features, [3, 5, 10, 15, 20, 30, 35, 50, 75]),
    feature_utils.get_lr_features,
    functools.partial(feature_utils.get_outlier_features, 1.5),
    feature_utils.get_quantiles
])

100%|██████████| 1686/1686 [00:05<00:00, 284.91it/s]


In [160]:
X_test, _ = feature_utils.construct_features_df_parallel(time_series_data_test, [
    functools.partial(feature_utils.get_fn_features, stats_features),
    functools.partial(feature_utils.get_fn_features, business_features),
    functools.partial(feature_utils.get_fourier_features, 15), 
    functools.partial(feature_utils.get_ma_features, [3, 5, 10, 15, 20, 30, 35, 50, 75]),
    feature_utils.get_lr_features,
    functools.partial(feature_utils.get_outlier_features, 1.5),
    feature_utils.get_quantiles
])

100%|██████████| 19826/19826 [00:22<00:00, 893.87it/s]


In [158]:
X = X.reset_index().rename(columns={'index':'id'})
X = X.merge(df_balanced, on='id')
y = X['target']
X = X.drop('target', axis=1)

In [161]:
X_test = X_test.reset_index().rename(columns={'index':'id'})
X_test = X_test.merge(remaining_data_test, on='id')
X_test

,id,mean,std,min,max,max_diff,50%,absolute_gain_basis,growth_coef_basis,average_gain_rate,...,feature_312,feature_330,feature_339,feature_341,feature_342,feature_372,feature_375,feature_383,feature_396,feature_411
0,38731,-0.306680,0.913272,-2.778084,2.320501,3.992224,-0.244131,0.191781,0.729252,0.004255,...,0,1,4,4,0,4,0,1,1,0
1,38732,-0.273013,0.872385,-2.998703,2.414822,3.596887,-0.256034,0.945115,-1.353244,0.004698,...,0,1,6,4,1,4,0,0,2,0
2,38733,-0.329177,0.911056,-2.966769,2.680878,4.238812,-0.344839,0.962365,-2.903910,0.003094,...,0,1,2,4,1,4,0,1,2,0
3,38734,-0.272798,0.932130,-3.115448,2.877415,4.607735,-0.251455,0.287336,0.425580,0.002750,...,0,1,2,4,1,4,0,0,2,0
4,38735,-0.232202,0.931777,-2.778429,2.214259,4.278078,-0.187652,1.777203,-0.821872,0.003579,...,0,1,4,4,1,4,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19821,56564,0.484713,1.095042,-2.545208,3.103672,4.622159,0.506542,-2.802310,-0.952826,0.000699,...,0,3,7,2,1,2,1,1,3,0
19822,56565,0.473438,1.129077,-2.428352,5.199338,6.422887,0.535983,0.085258,0.373105,0.006946,...,0,3,15,1,1,1,4,0,3,0
19823,56566,-0.288482,0.978456,-2.947679,2.953831,4.166893,-0.305527,0.359313,0.554098,-0.001617,...,0,1,4,4,1,4,0,0,1,0
19824,56567,-0.301437,0.924364,-2.832634,2.445034,3.983042,-0.267645,0.733009,2.169662,0.002631,...,0,1,4,4,1,4,0,0,1,0


Отбор признаков и кросс валидация

In [137]:
# selector = SelectKBest(f_classif, k=50)
# selector.fit(X, y)

# # Применяем маску отобранных признаков к X и X_test
# X = X.loc[:, selector.get_support()]
# X_test = X_test.loc[:, selector.get_support()]

In [162]:
X_train_CV, X_test_CV, y_train_CV, y_test_CV = train_test_split(X, y, test_size=0.33, random_state=42) #

Обучение модели

In [163]:
model_xgb = xgb.XGBClassifier(enable_categorical=True)

In [170]:
param_dist = {
    'n_estimators': [50, 100, 200, 300, 400, 500, 600, 700],
    'max_depth': [1, 2, 3, 5, 7, 10, 12, 15, 17, 20],
    'learning_rate': [0.001, 0.005, 0.01, 0.1, 0.2, 0.3],
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.3, 0.4, 0.5, 0.7],
    'colsample_bylevel': [0.3, 0.5, 0.7, 1.0],
    'min_child_weight': [1, 3, 5, 7, 10],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_lambda': [0, 1, 2, 3, 5, 10],
    # 'scale_pos_weight': list(np.arange(0.8, 2.2, 0.2))
}

random_search = RandomizedSearchCV(estimator=model_xgb,
                                        param_distributions=param_dist,
                                        n_iter=125,  
                                        scoring='roc_auc',   
                                        n_jobs=-1, 
                                        cv=3,  
                                        verbose=10,
                                        random_state=42)

random_search.fit(X_train_CV, y_train_CV)

Fitting 3 folds for each of 125 candidates, totalling 375 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=True,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=...
                   param_distributions={'colsample_bylevel': [0.3, 0.5, 0.7,
                                                              1.0],
                                        'colsample_bytree': [0.3, 0.4, 0.5,
                                                             0.7],
                                        'gamma': [0, 0.1, 0.2, 0.3],
                                        'learning_rate': [0.001, 0.005, 0.01,
                                                          0.1, 0.2, 0.3],
                                        'max_depth': [1, 2, 3, 5, 7, 10, 12, 15,
                                                      17, 20],
                                        'min_child_weight': [1, 3, 5, 7, 10],
                                        'n_estimators': [50, 100, 200, 300, 400,
                                                         500, 600, 700],
                                        'reg_lambda': [0, 1, 2, 3, 5, 10],
                                        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9,
                                                      1.0]},
                   random_state=42, scoring='roc_auc', verbose=10)

In [171]:
best_params = random_search.best_params_
best_score = random_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Cross-Validation Score: {best_score:.4f}")

Best Parameters: {'subsample': 0.7, 'reg_lambda': 0, 'n_estimators': 600, 'min_child_weight': 1, 'max_depth': 10, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.5}
Best Cross-Validation Score: 0.7329


Оценка точности модели

In [172]:
confusion_matrix(y_test_CV, random_search.predict(X_test_CV))

array([[187,  99],
       [ 71, 200]])

In [173]:
precision_score(y_test_CV, random_search.predict(X_test_CV))

np.float64(0.6688963210702341)

In [174]:
recall_score(y_test_CV, random_search.predict(X_test_CV))

np.float64(0.7380073800738007)

In [112]:
best_model = random_search.best_estimator_
y_pred = best_model.predict_proba(X_test_CV)

In [115]:
df_timeseries_T, remaining_data_T = processing.split_dataframe_by_condition(data_train, condition)
processing.df_processing(remaining_data_T, True)
df_transform = processing.df_transforming(df_timeseries_T)
time_series_data = processing.create_time_series_dict(df_transform)

d:\Study\DS\Hacatons\AlfaHack\processing.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('int')
d:\Study\DS\Hacatons\AlfaHack\processing.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('int')
d:\Study\DS\Hacatons\AlfaHack\processing.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [116]:
X_T, y_T = feature_utils.construct_features_df_parallel(time_series_data, [
    functools.partial(feature_utils.get_fn_features, stats_features),
    functools.partial(feature_utils.get_fn_features, business_features),
    functools.partial(feature_utils.get_fourier_features, 15), 
    functools.partial(feature_utils.get_ma_features, [3, 5, 10, 15, 20, 30, 35, 50, 75]),
    feature_utils.get_lr_features,
    functools.partial(feature_utils.get_outlier_features, 1.5),
    feature_utils.get_quantiles
])

100%|██████████| 38731/38731 [00:40<00:00, 948.88it/s] 


In [ ]:
X_T = X_T.reset_index().rename(columns={'index':'id'})
X_T = X_T.merge(remaining_data_train, on='id')
y_T = X_T['target']
X_T = X_T.drop('target', axis=1)

,id,mean,std,min,max,max_diff,50%,absolute_gain_basis,growth_coef_basis,average_gain_rate,...,feature_312,feature_330,feature_339,feature_341,feature_342,feature_372,feature_375,feature_383,feature_396,feature_411
0,4,-0.204561,0.891339,-3.093100,3.043426,4.809702,-0.207314,0.770943,-0.706395,-0.003122,...,0,3,1,4,0,4,0,0,3,0
1,61,0.153421,0.928425,-2.389160,2.613495,3.939532,0.165393,-0.419565,1.182384,-0.001598,...,0,3,1,4,0,4,0,0,3,0
2,70,-0.094642,1.004006,-2.937777,2.847850,4.637849,-0.060663,0.361668,-0.499444,0.001893,...,2,1,4,4,2,4,0,2,1,1
3,75,-0.165247,0.977498,-2.916903,2.583335,3.835610,-0.119008,1.152320,-0.709172,-0.002055,...,2,1,4,4,2,4,0,2,1,1
4,144,-0.320157,0.909921,-3.192508,2.263025,3.813637,-0.347967,-0.259050,0.344778,-0.001160,...,0,1,4,4,0,4,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,34680,-0.249998,0.879999,-2.857923,2.262654,3.391116,-0.253746,-1.083540,9.819433,-0.005584,...,0,1,6,4,1,4,0,0,2,0
1682,34768,-0.233932,0.968306,-3.103003,3.159050,4.379279,-0.232254,1.960272,-1.030979,0.002722,...,1,3,6,4,0,4,0,0,3,0
1683,34803,-0.146013,0.927470,-2.919625,2.556775,3.813853,-0.170065,-0.441216,2.146026,-0.007153,...,0,3,2,4,0,4,0,0,3,0
1684,34817,-0.178022,0.966535,-2.702844,3.092661,4.114359,-0.108863,-1.572431,-1.429105,-0.001607,...,0,3,6,4,1,4,0,0,3,0


In [62]:
X_result_testing = X_result_testing.reset_index().rename(columns={'index':'id'})
X_result_testing = X_result_testing.merge(data_train.drop('target', axis=1), on='id')
X_result_testing = X_result_testing.loc[:, selector.get_support()]


ValueError: The column label 'id' is not unique.

In [ ]:
roc_auc_score(y_result_testing, random_search.predict(X_result_testing))

np.float64(0.508027327469262)

In [52]:
y_result_testing.value_counts()

0    37888
1      843
Name: count, dtype: int64

In [ ]:
for x_t, x in zip(X_result_testing.columns, X_test.columns):
    print('F') if x_t != x else print('T')

T
T
T
T
T
T
T
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T


Предсказание модели

In [ ]:
random_search.predict_proba(X_test)

array([[0.12614387, 0.8738561 ],
       [0.33984178, 0.6601582 ],
       [0.67853314, 0.32146686],
       ...,
       [0.2664914 , 0.7335086 ],
       [0.34836257, 0.65163743],
       [0.317572  , 0.682428  ]], dtype=float32)

In [ ]:
y_test_pred = random_search.predict_proba(X_test)

y_test_pred = pd.Series(y_test_pred[:, 1])

data_test['target'] = y_test_pred

data_test[['id', 'target']].to_csv('baseline_submission_case1.csv', index=False)

In [ ]:
data_test[['id', 'target']]

,id,target
0,38731,0.008371
1,38732,0.993113
2,38733,0.006742
3,38734,0.007088
4,38735,0.009269
...,...,...
19821,56564,NaN
19822,56565,NaN
19823,56566,NaN
19824,56567,NaN


In [ ]:
prediction = pd.read_csv('baseline_submission_case1.csv')

In [ ]:
prediction['target'].isna().value_counts()

target
False    19826
Name: count, dtype: int64